In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# from main import get_vsa, get_similarity, get_test_data
# import matplotlib.pyplot as plt

# device = 'cuda'
# vsa = get_vsa(device)
# v1 = vsa.get_vector([(1,1,5,6),(2,0,2,5),(1,2,0,9)], quantize=True)
# print(get_similarity(v1, vsa.get_vector((2,0,2,3)), True))
# print(get_similarity(v1, vsa.get_vector((2,0,2,5)), True))
# dl = get_test_data(vsa)
# plt.imshow(dl.dataset[226][0])

In [ ]:
from tools.model_quantization import *
from datasets import get_test_data
from models.nn_non_decomposed import MultiConceptNonDecomposed
import torch
import os.path
from models.vsa import get_vsa
from main import get_transform, get_cos_similarity, get_dot_similarity

root = "./tests/HARDWARE-2048dim-128fd-3x-3y-7color/algo1"
pt = os.path.join(root, "model_weights_5objs_128batch_300000samples_20epoch_0.0182loss_11-07-06-14.pt")
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
model = MultiConceptNonDecomposed(dim=2048, device=device)
model.eval()    
checkpoint = torch.load(pt, map_location=device)
model.load_state_dict(checkpoint)
vsa = get_vsa(dir=root, mode="HARDWARE", algo="algo1", dim=2048, fold_dim=128, ehd_bits=8, device=device)

dl = get_test_data(root, vsa=vsa, shuffle=False, num_samples=1000, max_num_objects=5, single_count=True)
image, label, target, _ = dl.dataset[237]
infer_result = model(get_transform()(image.unsqueeze(0).to(device)))
(infer_result.round() == target).all()

for l in label:
    print(f"{l} cos:", get_cos_similarity(target, vsa.get_vector([l])))
    print(f"{l} dot quantized:", get_dot_similarity(target, vsa.get_vector([l]), quantized=False))
    print(f"{l} dot expand:", torch.matmul(target.type(torch.float32), vsa.get_vector([l]).type(torch.float32)))

v = (0,0,0,9)
print(f"{v} cos:", get_cos_similarity(target, vsa.get_vector([v])))
print(f"{v} dot quantized:", get_dot_similarity(target, vsa.get_vector([v]), quantized=False))
print(f"{v} dot expand:", torch.matmul(target.type(torch.float32), vsa.get_vector([v]).type(torch.float32)))


In [ ]:
merge_biases(model)
remove_biases(model)
test(model, dl, max_iter=100)

In [ ]:
stop_profiling = register_activation_profiling_hooks(model)
test(model, dl, max_iter=100)
stop_profiling()

In [ ]:
quantize_layer_weights(model)
quantize_activations(model)
quantize_bottleneck_layers(model)
quantize_averaging_layer(model)
quantize_layer_biases(model)

In [ ]:
test(model, dl, max_iter=100, quantized=True)

In [ ]:
for c in model.model.modules():
    print(c)

In [ ]:
import os
import torch
from models.nn_non_decomposed import MultiConceptNonDecomposed
from tools.model_quantization import profile_model
from tools.dump import generate_model_header, generate_actions
from datasets import get_test_data

root = "./tests/HARDWARE-1024dim-256fd-3x-3y-7color/algo1"
pt = os.path.join(root, "model_weights_3objs_128batch_20epoch_300000samples_0.0485loss_10-14-21-53.pt")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MultiConceptNonDecomposed(dim=1024, device=device)
model.eval()
checkpoint = torch.load(pt, map_location=device)
model.load_state_dict(checkpoint)
dl = get_test_data(root, shuffle=True, num_samples=300, max_num_objects=3)
profile_model(model, dl)
generate_model_header(model, gemmini_dim=16)
generate_actions(model)